# Model Artifact Memory Consumption Tests

In [319]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [320]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## MLServer sklearn Iris

In [321]:
numModels = 5
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

60375040


In [322]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"iris'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}


In [323]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

133799936


In [324]:
for round in range(2):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/iris$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: iris"$MODEL_NUM\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

env: MODEL_NUM=1
{"model_name":"iris1","model_version":"1","id":"7f99b31f-b6d5-4ada-9c4e-01a192410e9e","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}
200
env: MODEL_NUM=2
{"model_name":"iris2","model_version":"1","id":"f98a4dfa-5458-431c-a57a-1057742bd419","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}
200
env: MODEL_NUM=3
{"model_name":"iris3","model_version":"1","id":"9bf03f02-1c7e-4e43-8996-06e5094fda7e","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}
200
env: MODEL_NUM=4
{"model_name":"iris4","model_version":"1","id":"a5e9abf9-0783-4972-be49-09215f1b870b","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}
200
env: MODEL_NUM=5
{"model_name":"iris5","model_version":"1","id":"f38432be-5a55-448a-9744-ce351c9238fd","parameters":null,"outputs":[

In [325]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

527978496


In [326]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"iris'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}


In [327]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

136626176


In [328]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

78270464.0


## MLServer Xgboost Agaricus

In [329]:
numModels = 5
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

136503296


In [330]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"agaricus'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/mushroom",\
                           "requirements":["sklearn"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}


In [331]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

144171008


In [332]:
!grpcurl -d '{"model":{"name":"agaricus5"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "agaricus5",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T15:01:30.142909017Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T15:01:30.142909017Z"
      }
    }
  ]
}


In [333]:
for round in range(1):
    for x in range(1,numModels+1):
        print("")
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/agaricus$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: agaricus"$MODEL_NUM\
            -d '{"inputs": [{"name": "predict","shape": [1, 126],"datatype": "FP32","data":[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}]}'


env: MODEL_NUM=1
{"model_name":"agaricus1","model_version":"1","id":"b6d83cea-d7cc-4e2a-85bd-0cad534432b3","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[0.012412077747285366]}]}
200

env: MODEL_NUM=2
{"model_name":"agaricus2","model_version":"1","id":"897d05e6-63ce-4583-8214-e4a68ddd5c1e","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[0.012412077747285366]}]}
200

env: MODEL_NUM=3
{"model_name":"agaricus3","model_version":"1","id":"dc07304c-18d0-4a0b-b4f1-6cf30f856235","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[0.012412077747285366]}]}
200

env: MODEL_NUM=4
{"model_name":"agaricus4","model_version":"1","id":"bb567ad9-d7ed-4fed-9b46-8db011bd0ab6","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"FP32","parameters":null,"data":[0.012412077747285366]}]}
200

env: MODEL_NUM=5
{"model_name":"agaricu

In [334]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

535199744


In [335]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"agaricus'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}


In [336]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

146157568


In [337]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

77808435.2


## MLServer Conclusion

 * Around 80M is used per Model - both models tested were smallish 1K (disk size) models.

## Triton Simple Tensorflow Model

In [347]:
numModels = 20
memory=!kubectl exec seldon-triton-0 -n seldon-mesh -c triton -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

110080000


In [348]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"simple'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/triton/simple",\
                           "requirements":["tensorflow"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}
env: MODEL_NUM=6
{
  
}
env: MODEL_NUM=7
{
  
}
env: MODEL_NUM=8
{
  
}
env: MODEL_NUM=9
{
  
}
env: MODEL_NUM=10
{
  
}
env: MODEL_NUM=11
{
  
}
env: MODEL_NUM=12
{
  
}
env: MODEL_NUM=13
{
  
}
env: MODEL_NUM=14
{
  
}
env: MODEL_NUM=15
{
  
}
env: MODEL_NUM=16
{
  
}
env: MODEL_NUM=17
{
  
}
env: MODEL_NUM=18
{
  
}
env: MODEL_NUM=19
{
  
}
env: MODEL_NUM=20
{
  
}


In [349]:
for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !grpcurl -d '{"model":{"name":"simple'$MODEL_NUM'"}}' \
             -plaintext \
             -import-path ../../apis \
             -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus


env: MODEL_NUM=1
{
  "modelName": "simple1",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T15:37:10.942143768Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T15:37:10.942143768Z"
      }
    }
  ]
}
env: MODEL_NUM=2
{
  "modelName": "simple2",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T15:37:11.053989777Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T15:37:11.053989777Z"
      }
    }
  ]
}
env: MODEL_NUM=3
{
  "modelName": "simple3",
  "versions": [
    {
      "version": 1,
      "serverName": "

env: MODEL_NUM=20
{
  "modelName": "simple20",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T15:37:13.536461389Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T15:37:13.536461389Z"
      }
    }
  ]
}


In [350]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

146141184


In [351]:
for round in range(1):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/simple$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: simple"$MODEL_NUM\
            -d '{"inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}'

env: MODEL_NUM=1
{"model_name":"simple1","model_version":"1","outputs":[{"name":"OUTPUT0","datatype":"INT32","shape":[1,16],"data":[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32]},{"name":"OUTPUT1","datatype":"INT32","shape":[1,16],"data":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}]}
200
env: MODEL_NUM=2
{"model_name":"simple2","model_version":"1","outputs":[{"name":"OUTPUT0","datatype":"INT32","shape":[1,16],"data":[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32]},{"name":"OUTPUT1","datatype":"INT32","shape":[1,16],"data":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}]}
200
env: MODEL_NUM=3
{"model_name":"simple3","model_version":"1","outputs":[{"name":"OUTPUT0","datatype":"INT32","shape":[1,16],"data":[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32]},{"name":"OUTPUT1","datatype":"INT32","shape":[1,16],"data":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}]}
200
env: MODEL_NUM=4
{"model_name":"simple4","model_version":"1","outputs":[{"name":"OUTPUT0","datatype":"INT32","shape":[1,16],"data":[2,4,6,8,10,12,14,16,18,20,22,24,26

In [352]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

146161664


In [353]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"simple'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}
env: MODEL_NUM=6
{
  
}
env: MODEL_NUM=7
{
  
}
env: MODEL_NUM=8
{
  
}
env: MODEL_NUM=9
{
  
}
env: MODEL_NUM=10
{
  
}
env: MODEL_NUM=11
{
  
}
env: MODEL_NUM=12
{
  
}
env: MODEL_NUM=13
{
  
}
env: MODEL_NUM=14
{
  
}
env: MODEL_NUM=15
{
  
}
env: MODEL_NUM=16
{
  
}
env: MODEL_NUM=17
{
  
}
env: MODEL_NUM=18
{
  
}
env: MODEL_NUM=19
{
  
}
env: MODEL_NUM=20
{
  
}


In [354]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

146300928


In [355]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

-6963.2


## Triton XGBoost Agaricus Model

In [413]:
numModels = 20
memory=!kubectl exec seldon-triton-0 -n seldon-mesh -c triton -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryInitial=int(memory[0])
print(memoryInitial)

102055936


In [414]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{ \
              "meta":{"name":"triton-agaricus'$MODEL_NUM'"},\
              "modelSpec":{"uri":"gs://seldon-models/triton/agaricus",\
                           "requirements":["tensorflow"],\
                           "memoryBytes":1},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}
env: MODEL_NUM=6
{
  
}
env: MODEL_NUM=7
{
  
}
env: MODEL_NUM=8
{
  
}
env: MODEL_NUM=9
{
  
}
env: MODEL_NUM=10
{
  
}
env: MODEL_NUM=11
{
  
}
env: MODEL_NUM=12
{
  
}
env: MODEL_NUM=13
{
  
}
env: MODEL_NUM=14
{
  
}
env: MODEL_NUM=15
{
  
}
env: MODEL_NUM=16
{
  
}
env: MODEL_NUM=17
{
  
}
env: MODEL_NUM=18
{
  
}
env: MODEL_NUM=19
{
  
}
env: MODEL_NUM=20
{
  
}


In [415]:
for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !grpcurl -d '{"model":{"name":"triton-agaricus'$MODEL_NUM'"}}' \
             -plaintext \
             -import-path ../../apis \
             -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus


env: MODEL_NUM=1
{
  "modelName": "triton-agaricus1",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T16:19:10.420116632Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T16:19:10.420116632Z"
      }
    }
  ]
}
env: MODEL_NUM=2
{
  "modelName": "triton-agaricus2",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T16:19:10.554501173Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T16:19:10.554501173Z"
      }
    }
  ]
}
env: MODEL_NUM=3
{
  "modelName": "triton-agaricus3",
  "versions": [
    {
      "version

env: MODEL_NUM=19
{
  "modelName": "triton-agaricus19",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T16:19:12.939841458Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T16:19:12.939841458Z"
      }
    }
  ]
}
env: MODEL_NUM=20
{
  "modelName": "triton-agaricus20",
  "versions": [
    {
      "version": 1,
      "serverName": "triton",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2021-12-27T16:19:13.078842566Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2021-12-27T16:19:13.078842566Z"
      }
    }
  ]
}


In [416]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterLoad=int(memory[0])
print(memoryAfterLoad)

60731392


In [417]:
for round in range(1):
    for x in range(1,numModels+1):
        %env MODEL_NUM=$x
        !curl -w "\n%{http_code}\n" http://${MESH_IP}/v2/models/triton-agaricus$MODEL_NUM/infer -H "Content-Type: application/json" -H "Host: triton-agaricus"$MODEL_NUM\
            -d '{"inputs": [{"name": "input__0","shape": [1, 126],"datatype": "FP32","data":[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}]}'

env: MODEL_NUM=1
{"model_name":"triton-agaricus1","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env: MODEL_NUM=2
{"model_name":"triton-agaricus2","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env: MODEL_NUM=3
{"model_name":"triton-agaricus3","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env: MODEL_NUM=4
{"model_name":"triton-agaricus4","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env: MODEL_NUM=5
{"model_name":"triton-agaricus5","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env: MODEL_NUM=6
{"model_name":"triton-agaricus6","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[1],"data":[0.9875879287719727]}]}
200
env:

In [418]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterCurl=int(memory[0])
print(memoryAfterCurl)

60903424


In [419]:
for x in range(1,numModels+1):
    %env MODEL_NUM=$x
    !grpcurl -d '{"model":{"name":"triton-agaricus'$MODEL_NUM'"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel


env: MODEL_NUM=1
{
  
}
env: MODEL_NUM=2
{
  
}
env: MODEL_NUM=3
{
  
}
env: MODEL_NUM=4
{
  
}
env: MODEL_NUM=5
{
  
}
env: MODEL_NUM=6
{
  
}
env: MODEL_NUM=7
{
  
}
env: MODEL_NUM=8
{
  
}
env: MODEL_NUM=9
{
  
}
env: MODEL_NUM=10
{
  
}
env: MODEL_NUM=11
{
  
}
env: MODEL_NUM=12
{
  
}
env: MODEL_NUM=13
{
  
}
env: MODEL_NUM=14
{
  
}
env: MODEL_NUM=15
{
  
}
env: MODEL_NUM=16
{
  
}
env: MODEL_NUM=17
{
  
}
env: MODEL_NUM=18
{
  
}
env: MODEL_NUM=19
{
  
}
env: MODEL_NUM=20
{
  
}


In [420]:
memory=!kubectl exec seldon-mlserver-0 -n seldon-mesh -c mlserver -- cat /sys/fs/cgroup/memory/memory.usage_in_bytes
memoryAfterUnload=int(memory[0])
print(memoryAfterUnload)

60866560


In [421]:
memoryPerModel = ((memoryAfterCurl - memoryAfterUnload)/numModels) 
print(memoryPerModel)

1843.2


## Triton Conclusions

 * Triton models take up variable small amount 0-10K for a small model of around 300 bytes on disk
   * Sometimes the memory used is negative by above calulation
   * Maybe due to delayed memory garbage collection?